In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

PROJECT_ROOT = r"C:\Users\kewtiiiii\Desktop\Projects\engine-cycle-forecast"

raw_path       = os.path.join(PROJECT_ROOT, "data", "raw", "train_FD001.txt")
processed_path = os.path.join(PROJECT_ROOT, "data", "processed", "fd001_processed.csv")
db_path        = os.path.join(PROJECT_ROOT, "database", "nasa_engines.db")
metrics_path   = os.path.join(PROJECT_ROOT, "artifacts", "metrics", "rf_performance.csv")
model_path     = os.path.join(PROJECT_ROOT, "artifacts", "models", "rf_rul_model.pkl")
figures_path   = os.path.join(PROJECT_ROOT, "artifacts", "figures")
os.makedirs(os.path.join(PROJECT_ROOT, "database"), exist_ok=True)


os.makedirs(os.path.join(PROJECT_ROOT, "data", "processed"), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, "artifacts", "metrics"), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, "artifacts", "models"), exist_ok=True)
os.makedirs(figures_path, exist_ok=True)

df = pd.read_csv(raw_path, sep=r"\s+", header=None)


In [2]:
cols = ["unit", "cycle"] + [f"op_setting_{i}" for i in range(1,4)] + [f"sensor_{i}" for i in range(1,22)]
df.columns = cols

missing_counts = df.isnull().sum()
print("Missing values per column:\n", missing_counts)

constant_cols = [col for col in df.columns if df[col].nunique() == 1]
print("Constant columns:", constant_cols)

desc_stats = df.describe()
desc_stats.to_csv(os.path.join(PROJECT_ROOT, "artifacts", "metrics", "descriptive_stats.csv"))

plt.figure(figsize=(15,12))
sns.heatmap(df.corr(), cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.savefig(os.path.join(figures_path, "correlation_heatmap.png"), dpi=300, bbox_inches="tight")
plt.close()

print("✅ EDA completed.")

Missing values per column:
 unit            0
cycle           0
op_setting_1    0
op_setting_2    0
op_setting_3    0
sensor_1        0
sensor_2        0
sensor_3        0
sensor_4        0
sensor_5        0
sensor_6        0
sensor_7        0
sensor_8        0
sensor_9        0
sensor_10       0
sensor_11       0
sensor_12       0
sensor_13       0
sensor_14       0
sensor_15       0
sensor_16       0
sensor_17       0
sensor_18       0
sensor_19       0
sensor_20       0
sensor_21       0
dtype: int64
Constant columns: ['op_setting_3', 'sensor_1', 'sensor_5', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']
✅ EDA completed.


In [ ]:
# -----------------------------
# Sensor degradation
# -----------------------------
sample_engines = df["unit"].unique()[:5]  
sensor_to_plot = "sensor_1"  

plt.figure(figsize=(12,6))
for eng in sample_engines:
    subset = df[df["unit"]==eng]
    plt.plot(subset["cycle"], subset[sensor_to_plot], label=f"Engine {eng}")

plt.xlabel("Cycle")
plt.ylabel(sensor_to_plot)
plt.title(f"{sensor_to_plot} Degradation Over Cycles (Sample Engines)")
plt.legend()
plt.savefig(os.path.join(figures_path, f"{sensor_to_plot}_degradation.png"), dpi=300, bbox_inches="tight")
plt.close()
print(f"✅ {sensor_to_plot} degradation plot saved.")


✅ sensor_1 degradation plot saved.


In [4]:
max_cycles = df.groupby("unit")["cycle"].max().reset_index()
max_cycles.columns = ["unit", "max_cycle"]
df = df.merge(max_cycles, on="unit")
df["rul"] = df["max_cycle"] - df["cycle"]
df.drop(columns=["max_cycle"], inplace=True)

df.to_csv(processed_path, index=False)


rul_corr = df.corr()["rul"].sort_values(ascending=False)
print("Top correlations with RUL:\n", rul_corr.head(10))


Top correlations with RUL:
 rul             1.000000
sensor_12       0.671983
sensor_7        0.657223
sensor_21       0.635662
sensor_20       0.629428
unit            0.078753
op_setting_2   -0.001948
op_setting_1   -0.003198
sensor_6       -0.128348
sensor_14      -0.306769
Name: rul, dtype: float64


In [5]:
features = [c for c in df.columns if c != "rul"]
X = df[features]
y = df["rul"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

search = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=12, cv=3,
    scoring='neg_mean_squared_error', n_jobs=-1, random_state=42, verbose=1
)

search.fit(X_train, y_train)
best_rf = search.best_estimator_

y_pred = best_rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2  = r2_score(y_test, y_pred)
pd.DataFrame([{"MSE": mse, "R2": r2}]).to_csv(metrics_path, index=False)

joblib.dump(best_rf, model_path)
print("✅ Trained RF model saved.")



Fitting 3 folds for each of 12 candidates, totalling 36 fits
✅ Trained RF model saved.


In [7]:
plt.figure(figsize=(10,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([0,max(y_test)], [0,max(y_test)], 'r--')
plt.xlabel("Actual RUL")
plt.ylabel("Predicted RUL")
plt.title("Predicted vs Actual RUL")
plt.savefig(os.path.join(figures_path, "pred_vs_actual.png"), dpi=300, bbox_inches="tight")
plt.close()
print("✅ Predicted vs Actual plot saved.")


✅ Predicted vs Actual plot saved.


In [8]:
residuals = y_test - y_pred
plt.figure(figsize=(10,6))
plt.scatter(y_test, residuals, alpha=0.4)
plt.axhline(0, linestyle='--')
plt.xlabel("Actual RUL")
plt.ylabel("Residual")
plt.title("Residual Analysis")
plt.savefig(os.path.join(figures_path, "residuals.png"), dpi=300, bbox_inches="tight")
plt.close()
print("✅ Residual plot saved.")

✅ Residual plot saved.


In [9]:
pred_df = X_test.copy()
pred_df["actual_rul"] = y_test
pred_df["predicted_rul"] = y_pred
pred_df.to_csv(os.path.join(PROJECT_ROOT, "artifacts", "metrics", "predictions_sample.csv"), index=False)
print("✅ Predictions sample saved.")

✅ Predictions sample saved.


In [10]:
importances = best_rf.feature_importances_
feat_importance_df = pd.DataFrame({
    "feature": features,
    "importance": importances
}).sort_values(by="importance", ascending=False)

# Save feature importance CSV
feat_importance_df.to_csv(os.path.join(PROJECT_ROOT, "artifacts", "metrics", "feature_importance.csv"), index=False)
print("✅ Feature importance CSV saved.")

# Plot top 10 features
plt.figure(figsize=(10,6))
sns.barplot(x="importance", y="feature", data=feat_importance_df.head(10))
plt.title("Top 10 Feature Importances")
plt.tight_layout()
plt.savefig(os.path.join(figures_path, "feature_importance.png"), dpi=300)
plt.close()
print("✅ Feature importance plot saved.")


✅ Feature importance CSV saved.
✅ Feature importance plot saved.


In [11]:
import sqlite3
os.makedirs(os.path.join(PROJECT_ROOT, "database"), exist_ok=True)

conn = sqlite3.connect(db_path)

df.to_sql("fd001_processed", conn, if_exists="replace", index=False)

pred_df_with_unit = pred_df.copy()
pred_df_with_unit["unit"] = X_test["unit"].values
pred_df_with_unit.to_sql("fd001_predictions", conn, if_exists="replace", index=False)

conn.close()
print("✅ Data and predictions saved to SQLite database.")




✅ Data and predictions saved to SQLite database.


In [12]:
import joblib
joblib.dump(best_rf, "rf_rul_model.pkl")


['rf_rul_model.pkl']